In [ ]:
# 금가격, 국채가격, 변동성지수를 사용해 트리모델을 먼저 훈련, 
# 해당 모델에 기반해 다른 트리를 만듬 
# 이 과정을 순차적으로 계속하게 되면 편향이 적은 모델이 만들어짐.. 
# 만약 투자에 영향을 가장 많이 미치는것이 신문이나 방송등 뉴스 정보라고 생각한다면
# NLP기술을 활용해 데이터를 정제할수있음
# 핵심은 예측하고자 하는 시장의 움직임에 영향을 주거나 관련이 있다고 생각하되는 자료를 수집, 
# 가공하고 '이해를 바탕으로 한 ' 머신러닝 알고리즘을 적용해야한다는것임..

In [3]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import datetime
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance 
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import accuracy_score
from sklearn import svm 
import seaborn as sns; sns.set()

In [4]:
df = pd.read_csv('./data/ETFs_main.csv')

In [9]:
df.head()

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.20,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.07,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.42,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.79,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.95,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960


In [11]:
# 기술 지표 만들기
def moving_average(df, n):
    MA = pd.Series(df['CLOSE_SPY'].rolling(n, min_periods = n).mean(),name='MA_'+\
                  str(n))
    df = df.join(MA)
    return df 

def volume_moving_average(df,n):
    #거래량 이동평균 
    VMA = pd.Series(df['VOLUME'].rolling(n, min_periods=n).mean(), name='VMA_'+str(n))
    df = df.join(VMA)
    return df 

def relative_strength_index(df, n):
    """ 
    Calculate Relative Strength Index(RSI) for given data.

    :param df: pandas.DataFrame
    :param n:
    :return: pandas.DataFrame
    """
    i =0 

    UpI = [0]
    DoI = [0]
    while i+1 <=df.index[-1]:
        UpMove = df.loc[i+1,'HIGH'] - df.loc[i,'HIGH']
        DoMove = df.loc[i,'LOW'] - df.loc[i+1,'LOW']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            DoD = 0 
        DoI.append(DoD)
        i = i +1 
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
    RSI = pd.Series(PosDI/ (PosDI + NegDI), name = 'RSI_'+str(n))
    df = df.join(RSI)
    return df 

In [742]:
gold_etfs= ['GLD','IAU','GLDM','SGOL','IAUM','BAR','OUNZ','AAAU','DGL']

In [748]:
df_nas = fdr.StockListing('NASDAQ')
df_nas

100%|█████████████████████████████████████████████████████████████████████████████| 4621/4621 [00:13<00:00, 349.72it/s]


,Symbol,Name,Industry,IndustryCode
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010
2,AMZN,Amazon.com Inc,다양한 소매업,534020
3,TSLA,Tesla Inc,자동차 및 자동차 부품,531010
4,GOOGL,Alphabet Inc Class A,소프트웨어 및 IT서비스,572010
...,...,...,...,...
4616,RFACR,RF Acquisition Rights Exp 1st May 2028,기타 금융업,556010
4617,RWODR,Redwoods Acquisition Rights,기타 금융업,556010
4618,SVRE,Saverone 2014 Ltd ADR,소프트웨어 및 IT서비스,572010
4619,IVCAU,Investcorp India Acquisition Units,기타 금융업,556010


In [749]:
def nametocode_nas(name):
    return df_nas[df_nas.Symbol == name]['IndustryCode'].values[0]

In [750]:
gd_etfs=[]
for i in gold_etfs:
    gd_etfs.append(nametocode_nas(i))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [752]:
df_nas[df_nas.Symbol == 'GLD']['IndustryCode'].values[0]

Series([], Name: IndustryCode, dtype: object)

In [756]:
df_nys = fdr.StockListing('NYSE')
df_nys

100%|█████████████████████████████████████████████████████████████████████████████| 3447/3447 [00:07<00:00, 490.66it/s]


,Symbol,Name,Industry,IndustryCode
0,JNJ,Johnson & Johnson,제약,562010
1,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020
2,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010
3,WMT,Walmart Inc,식품 및 약품소매,543010
4,BRK.B,Berkshire Hathaway Inc Class B,복합 기업,544010
...,...,...,...,...
3442,WBS PR G,Webster Financial Depositary Shs Each Rep 1 40...,은행,551010
3443,KMPB,Kemper 5 875 Fixed Rate Reset Junior Subordina...,보험,553010
3444,ENOV WI,Colfax Corp,"건설장비 및 대형차, 기차, 배",521020
3445,ESAB WI,ESAB Corp,"건설장비 및 대형차, 기차, 배",521020


In [757]:
df_amx = fdr.StockListing('AMEX')
df_amx

100%|███████████████████████████████████████████████████████████████████████████████| 307/307 [00:00<00:00, 607.38it/s]


,Symbol,Name,Industry,IndustryCode
0,LNG,Cheniere Energy Inc,오일 및 가스 장비 및 서비스,501030
1,IMO,Imperial Oil Ltd,오일 및 가스,501020
2,CQP,Cheniere Energy Partners Units,오일 및 가스 장비 및 서비스,501030
3,CBOE,Cboe Global Markets Inc,투자은행 및 서비스,551020
4,PHYS,Sprott Physical Gold Trust,다양한 금융서비스,555010
...,...,...,...,...
302,GLV RT WI,Clough Global Dividend and Income Fund,다양한 금융서비스,555010
303,GLO RT WI,Clough Global Opportunities Fund,다양한 금융서비스,555010
304,TELZ,Tellurian 8 25 Senior Notes due 2028,오일 및 가스,501020
305,NHS RT,Neuberger Berman High Yield Strategies Fund Ri...,다양한 금융서비스,555010


In [758]:
df_sp = fdr.StockListing('SP500')
df_sp

C:\Users\bitcamp\anaconda3\lib\site-packages\FinanceDataReader\wikipedia\listing.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('\.', '')


,Symbol,Name,Sector,Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Pharmaceuticals
4,ABMD,Abiomed,Health Care,Health Care Equipment
...,...,...,...,...
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants
500,ZBRA,Zebra,Information Technology,Electronic Equipment & Instruments
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment
502,ZION,Zions Bancorp,Financials,Regional Banks


In [ ]:
df_nas[df_nas.Symbol == 'GLD']['IndustryCode'].values[0]

In [742]:
gold_etfs= ['GLD','IAU','GLDM','SGOL','IAUM','BAR','OUNZ','AAAU','DGL']

In [748]:
df_nas = fdr.StockListing('NASDAQ')
df_nas

100%|█████████████████████████████████████████████████████████████████████████████| 4621/4621 [00:13<00:00, 349.72it/s]


,Symbol,Name,Industry,IndustryCode
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010
2,AMZN,Amazon.com Inc,다양한 소매업,534020
3,TSLA,Tesla Inc,자동차 및 자동차 부품,531010
4,GOOGL,Alphabet Inc Class A,소프트웨어 및 IT서비스,572010
...,...,...,...,...
4616,RFACR,RF Acquisition Rights Exp 1st May 2028,기타 금융업,556010
4617,RWODR,Redwoods Acquisition Rights,기타 금융업,556010
4618,SVRE,Saverone 2014 Ltd ADR,소프트웨어 및 IT서비스,572010
4619,IVCAU,Investcorp India Acquisition Units,기타 금융업,556010


In [749]:
def nametocode_nas(name):
    return df_nas[df_nas.Symbol == name]['IndustryCode'].values[0]

In [750]:
gd_etfs=[]
for i in gold_etfs:
    gd_etfs.append(nametocode_nas(i))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [752]:
df_nas[df_nas.Symbol == 'GLD']['IndustryCode'].values[0]

Series([], Name: IndustryCode, dtype: object)

In [756]:
df_nys = fdr.StockListing('NYSE')
df_nys

100%|█████████████████████████████████████████████████████████████████████████████| 3447/3447 [00:07<00:00, 490.66it/s]


,Symbol,Name,Industry,IndustryCode
0,JNJ,Johnson & Johnson,제약,562010
1,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020
2,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010
3,WMT,Walmart Inc,식품 및 약품소매,543010
4,BRK.B,Berkshire Hathaway Inc Class B,복합 기업,544010
...,...,...,...,...
3442,WBS PR G,Webster Financial Depositary Shs Each Rep 1 40...,은행,551010
3443,KMPB,Kemper 5 875 Fixed Rate Reset Junior Subordina...,보험,553010
3444,ENOV WI,Colfax Corp,"건설장비 및 대형차, 기차, 배",521020
3445,ESAB WI,ESAB Corp,"건설장비 및 대형차, 기차, 배",521020


In [757]:
df_amx = fdr.StockListing('AMEX')
df_amx

100%|███████████████████████████████████████████████████████████████████████████████| 307/307 [00:00<00:00, 607.38it/s]


,Symbol,Name,Industry,IndustryCode
0,LNG,Cheniere Energy Inc,오일 및 가스 장비 및 서비스,501030
1,IMO,Imperial Oil Ltd,오일 및 가스,501020
2,CQP,Cheniere Energy Partners Units,오일 및 가스 장비 및 서비스,501030
3,CBOE,Cboe Global Markets Inc,투자은행 및 서비스,551020
4,PHYS,Sprott Physical Gold Trust,다양한 금융서비스,555010
...,...,...,...,...
302,GLV RT WI,Clough Global Dividend and Income Fund,다양한 금융서비스,555010
303,GLO RT WI,Clough Global Opportunities Fund,다양한 금융서비스,555010
304,TELZ,Tellurian 8 25 Senior Notes due 2028,오일 및 가스,501020
305,NHS RT,Neuberger Berman High Yield Strategies Fund Ri...,다양한 금융서비스,555010


In [758]:
df_sp = fdr.StockListing('SP500')
df_sp

C:\Users\bitcamp\anaconda3\lib\site-packages\FinanceDataReader\wikipedia\listing.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('\.', '')


,Symbol,Name,Sector,Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Pharmaceuticals
4,ABMD,Abiomed,Health Care,Health Care Equipment
...,...,...,...,...
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants
500,ZBRA,Zebra,Information Technology,Electronic Equipment & Instruments
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment
502,ZION,Zions Bancorp,Financials,Regional Banks


In [ ]:
df_nas[df_nas.Symbol == 'GLD']['IndustryCode'].values[0]

### 상위 10개 etf 도전

In [723]:
# lis = ['KODEX 200','TIGER 차이나전기차SOLACTIVE','KODEX 200선물인버스2X','KODEX 레버리지','TIGER 미국나스닥100',\
#        'KODEX 단기채권PLUS','TIGER 단기통안채','TIGER 200','TIGER 미국S&P500','KODEX 단기채권']

lis = ['KODEX 200','KODEX 레버리지','TIGER 미국나스닥100',\
       'TIGER 단기통안채','TIGER 200','KODEX 단기채권']
# 상장된지얼마안된거없앰

In [724]:
li = []
for i in lis:
    li.append(nametocode(i))

In [725]:
li

['069500', '122630', '133690', '157450', '102110', '153130']

In [639]:
# # df = fdr.DataReader('069500','2020')
# tmp = []
# for i in li:
#     a = fdr.DataReader(i,'2021')
#     tmp.append(a)

In [633]:
# 머지를 하는데 , 가로로하고 , 종목 코드를 컬럼 이름으로 ! 

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,28128,28197,27755,27812,5074135,-0.010566
2020-01-03,28053,28207,27654,27808,8408726,-0.000144
2020-01-06,27607,27688,27533,27595,6104793,-0.007660
2020-01-07,27774,27976,27708,27873,5862812,0.010074
2020-01-08,27654,27840,27509,27695,6771062,-0.006386
...,...,...,...,...,...,...
2022-05-03,35380,35645,35290,35290,5987778,-0.002403
2022-05-04,35450,35570,35245,35375,5560940,0.002409
2022-05-06,35010,35010,34785,34885,6080928,-0.013852


In [691]:
df_etfs = pd.read_csv('./data/ETFs_main.csv')
df_etfs

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.200,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.070,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.420,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.790,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.950,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766,2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.180
2767,2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.870
2768,2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.320
2769,2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.900


In [739]:
tmp_df = pd.DataFrame()
for i in li:
    a = fdr.DataReader(i,'2013')
    tmp_df = pd.concat([tmp_df,a],axis=1)
    tmp_df = tmp_df.rename(columns = {'Open':'Open_'+i,'High':'High_'+i,'Low':'Low_'+i,'Close':'Close_'+i,'Volume':'Volume_'+i,\
                            'Change':'Change_'+i})    

In [741]:
tmp_df

,Open_069500,High_069500,Low_069500,Close_069500,Volume_069500,Change_069500,Open_122630,High_122630,Low_122630,Close_122630,...,Low_102110,Close_102110,Volume_102110,Change_102110,Open_153130,High_153130,Low_153130,Close_153130,Volume_153130,Change_153130
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,22652,22962,22652,22939,5157531,0.019556,13175,13515,13170,13495,...,22187,22582,1574236,0.018859,89635,89640,89631,89643,135682,0.000156
2013-01-03,23077,23138,22846,22858,9339160,-0.003531,13700,13725,13400,13400,...,22474,22484,1329891,-0.004340,89659,89667,89654,89670,59047,0.000301
2013-01-04,22868,22885,22668,22719,6046477,-0.006081,13375,13425,13160,13195,...,22310,22372,1738859,-0.004981,89670,89684,89670,89688,49921,0.000201
2013-01-07,22727,22760,22549,22733,5262472,0.000616,13210,13265,13025,13220,...,22215,22409,901967,0.001654,89689,89689,89684,89692,47210,0.000045
2013-01-08,22609,22699,22476,22492,4643255,-0.010601,13135,13200,12955,12960,...,22159,22176,1282550,-0.010398,89694,89698,89689,89692,17457,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-03,35380,35645,35290,35290,5987778,-0.002403,19000,19280,18895,18930,...,35340,35370,637211,-0.001271,103360,103375,103355,103355,4274833,-0.000193
2022-05-04,35450,35570,35245,35375,5560940,0.002409,19080,19205,18855,18980,...,35290,35420,379994,0.001414,103370,103390,103370,103390,1963120,0.000339
2022-05-06,35010,35010,34785,34885,6080928,-0.013852,18570,18585,18355,18475,...,34840,34945,1204643,-0.013411,103375,103390,103375,103380,1574447,-0.000097


In [734]:
# 거래량 상위 10개
etfs_10 = ['KODEX 200선물인버스2X','KODEX 코스닥150선물인버스','KODEX 인버스','KODEX 레버리지','KODEX 코스닥150레버리지',\
           'TIGER 200선물인버스2X','KODEX WTI원유선물인버스(H)','TIGER 원유선물인버스(H)','KODEX 코스닥150','KODEX 200']

etfs_10 = ['KODEX 코스닥150선물인버스','KODEX 인버스','KODEX 레버리지','KODEX 코스닥150레버리지',\
           'TIGER 200선물인버스2X','KODEX WTI원유선물인버스(H)','TIGER 원유선물인버스(H)','KODEX 코스닥150','KODEX 200']

In [712]:
tmp_li=[]
for i in etfs_10:
    tmp_li.append(nametocode(i))

In [713]:
tmp_li

['252670',
 '251340',
 '114800',
 '122630',
 '233740',
 '252710',
 '271050',
 '217770',
 '229200',
 '069500']

In [720]:
tmp_df2 = pd.DataFrame()
for i in tmp_li:
    a = fdr.DataReader(i,'2015')
    tmp_df2 = pd.concat([tmp_df2,a],axis=1)
    tmp_df2 = tmp_df2.rename(columns = {'Open':'Open_'+i,'High':'High_'+i,'Low':'Low_'+i,'Close':'Close_'+i,'Volume':'Volume_'+i,\
                            'Change':'Change_'+i})    

In [721]:
tmp_df2.isnull().sum()

Open_252670      424
High_252670      424
Low_252670       424
Close_252670     424
Volume_252670    424
Change_252670    425
Open_251340      397
High_251340      397
Low_251340       397
Close_251340     397
Volume_251340    397
Change_251340    398
Open_114800        0
High_114800        0
Low_114800         0
Close_114800       0
Volume_114800      0
Change_114800      0
Open_122630        0
High_122630        0
Low_122630         0
Close_122630       0
Volume_122630      0
Change_122630      0
Open_233740      239
High_233740      239
Low_233740       239
Close_233740     239
Volume_233740    239
Change_233740    240
Open_252710      424
High_252710      424
Low_252710       424
Close_252710     424
Volume_252710    424
Change_252710    425
Open_271050      602
High_271050      602
Low_271050       602
Close_271050     602
Volume_271050    602
Change_271050    603
Open_217770       80
High_217770       80
Low_217770        80
Close_217770      80
Volume_217770     80
Change_217770